In [2]:
# Here i plan to neaten up the section that does the npv calulations incoprating the first and seoccnd payment dates varibles into the exsiting if statments from version 5.
import pandas as pd
import datetime
import numpy as np
import os
import matplotlib
from datetime import date
from dateutil.relativedelta import relativedelta
%matplotlib inline
os.chdir('C:/Users/harry/Desktop/Python/Ground_Rent_Pricing_Model/Data')

In [3]:
z1 = pd.read_csv('Tradeweb_FTSE_ClosePrices (convential).csv')
del z1['Close of Business Date']
del z1['Gilt Name']
del z1['ISIN']
del z1['Type']
del z1['Coupon']
del z1['Clean Price'] 
del z1 ['Dirty Price']
del z1['Mod Duration']
del z1['Accrued Interest']
z1 = z1.drop_duplicates()
z1['Maturity'] = pd.to_datetime(z1['Maturity'], format='%d/%m/%Y')
z1 = z1.set_index(pd.DatetimeIndex(z1['Maturity']))
z1 = z1.resample('d')
z1 = z1.mean()
z2 = z1.interpolate(method = 'polynomial', order = 1)


In [4]:
# This section is for calculating the npv of ground rent payments depending on the size, frequncey and any factor increases of payments
Freq_check = input('Are you expecting ground rent payments once or twice a year (enter 1 or 2): ')
if Freq_check == '1':
    first_payment_date = input('Please input the day you expect to receive your first ground rent payment (DD/MM/YYYY): ')
    final_payment = input('Please enter the date at which you will stop receiving payment (DD/MM/YYYY): ')
    risk_premium = input('Please enter the risk premium you would like to associate with this freehold investment, enter 10% as 10 or 9.5% as 9.5: ')
    risk_premium = float(risk_premium)
    end_date = z2.last_valid_index()
    first_payment_date = pd.to_datetime(first_payment_date, format='%d/%m/%Y')
    end_date = pd.to_datetime(end_date, format='%d/%m/%Y')
    df1 = pd.date_range(start = first_payment_date, end = end_date, freq = '12MS' )
    df2 = z2.loc[df1, 'Yield']
    df2 = df2.to_frame(name = 'Payments Batch #1')
    final_payment = pd.to_datetime(final_payment, format='%d/%m/%Y')
    v1 = df2.last_valid_index()
    futher_dates = pd.date_range(start = v1, end = final_payment, freq = '12MS')
    futher_dates = pd.DataFrame(futher_dates, index= futher_dates)
    v2 = float(df2.loc[v1, 'Payments Batch #1'])
    futher_dates['Payments Batch #1'] = v2
    del futher_dates[0]
    futher_dates = futher_dates.iloc[1:,:]
    df2 = df2.append(futher_dates)
    GR1 = input('Please input the value of the first ground rent payment (do not include commas): ')
    df2.insert(1, 'Payments', int(GR1))
    GRincrease = input('Are your ground rent payments set to increase at some point in the future?(yes/no) ')
    if GRincrease == 'no':
        print (*'No ground rent payment increase noted')
    elif GRincrease == 'yes':
        factor = input('Please enter the factor by which your ground rents are set to increase: ' )
        first_double_date =input('please enter the date of you will receive your first increased ground rent (DD/MM/YYYY): ')
        first_double_date = pd.to_datetime(first_double_date, format='%d/%m/%Y')
        end_of_double = df2.last_valid_index()
        date_range1 = pd.date_range(start = first_double_date, end = end_of_double, freq = '12MS')
        df2.loc[date_range1, 'Payments'] = df2.loc[date_range1, 'Payments'] * int(factor)
        inc_again = input('Do the ground rent payments increase again (yes/no)?')
        if inc_again == 'no':
            print ('No futher increases noted')
        elif inc_again == 'yes':
            second_factor = input('Please enter the factor by which your already increased ground rents are set to increase by: ' )
            second_increase_date = input('please enter the date of you will receive your second increased ground rent payments (DD/MM/YYYY): ')
            second_increase_date = pd.to_datetime(second_increase_date, format='%d/%m/%Y')
            end_of_double = df2.last_valid_index()
            date_range3 = pd.date_range(start = second_increase_date, end = end_of_double, freq = '12MS')
            df2.loc[date_range3, 'Payments'] = df2.loc[date_range3, 'Payments'] * float(second_factor)
    df2['Payments Batch #1'] = df2['Payments Batch #1']  + risk_premium
    df2['Payments'] = df2['Payments'].astype(int)
    num = df2['Payments'].tolist()
    rate_lists = df2['Payments Batch #1'].tolist()
    def npv(num, rate_lists):
        result = 0
        for t,(num,rate_lists) in enumerate(zip(num,rate_lists),1):
            result += (num / (1+(rate_lists/100)) ** t)
        return result
    npv1 = npv(num,rate_lists)
    print (npv1)
    npv2 = 0
elif Freq_check == '2':
    first_payment_date = input('Please input the day you expect to receive your first ground rent payment (DD/MM/YYYY): ')
    second_payment_date = input('Please input the day you expect to reveive your second ground rent payment (DD/MM/YYYY):')
    risk_premium = input('Please enter the risk premium you would like to associate with this freehold investment, enter 10% as 10 or 9.5% as 9.5: ')
    risk_premium = float(risk_premium)
    end_date = z2.last_valid_index()
    first_payment_date = pd.to_datetime(first_payment_date, format='%d/%m/%Y')
    second_payment_date = pd.to_datetime(second_payment_date, format='%d/%m/%Y')
    end_date = pd.to_datetime(end_date, format='%d/%m/%Y')
    df1 = pd.date_range(start = first_payment_date, end = end_date, freq = '12MS' )
    df2 = z2.loc[df1, 'Yield']
    df2 = df2.to_frame(name = 'Payments Batch #1')
    final_payment = input('Please enter the date at which you will stop receiving payment (DD/MM/YYYY): ')
    final_payment = pd.to_datetime(final_payment, format='%d/%m/%Y')
    v1 = df2.last_valid_index()
    futher_dates = pd.date_range(start = v1, end = final_payment, freq = '12MS')
    futher_dates = pd.DataFrame(futher_dates, index= futher_dates)
    v2 = float(df2.loc[v1, 'Payments Batch #1'])
  
    futher_dates['Payments Batch #1'] = v2
    del futher_dates[0]
    futher_dates = futher_dates.iloc[1:,:]
    df2 = df2.append(futher_dates)
    df3 = pd.date_range(start = second_payment_date, end = end_date, freq = '12MS' )
    df4 = z2.loc[df3, 'Yield']
    df4 = df4.to_frame(name = 'Payments Batch #2')
    final_payment = pd.to_datetime(final_payment, format='%d/%m/%Y')
    v1 = df4.last_valid_index()
    futher_dates = pd.date_range(start = v1, end = final_payment, freq = '12MS')
    futher_dates = pd.DataFrame(futher_dates, index= futher_dates)
    v2 = float(df4.loc[v1, 'Payments Batch #2'])
    futher_dates['Payments Batch #2'] = v2
    del futher_dates[0]
    futher_dates = futher_dates.iloc[1:,:]
    df4 = df4.append(futher_dates)
    GR1 = input('Please input the value of the first ground rent payment: ')
    GR2 = input('Please input the value of the first ground rent payment: ')
    df2.insert(1, 'Payments', int(GR1))
    df4.insert(1, 'Payments', int(GR2))
    GRincrease = input('Are your ground rent payments set to increase at some point in the future?(yes/no) ')
    if GRincrease == 'no':
        print (*'No ground rent payment increase noted')
    elif GRincrease == 'yes':
        factor = input('Please enter the factor by which your ground rents are set to increase: ' )
        first_double_date = input('Please enter the date of you will receive your first increased ground rent, make sure this aligns with the orginal payment schedule and is in the form (DD/MM/YYYY): ')
        first_double_date = pd.to_datetime(first_double_date, format='%d/%m/%Y')
        end_of_double = df2.last_valid_index()
        date_range1 = pd.date_range(start = first_double_date, end = end_of_double, freq = '12MS')
        df2.loc[date_range1, 'Payments'] = df2.loc[date_range1, 'Payments'] * int(factor)

        second_double_date = input('Please enter the date of you will receive your second increased ground rent payment, make sure this aligns with the orginal payment schedule and is in the form (DD/MM/YYYY): ')
        second_double_date = pd.to_datetime(second_double_date, format='%d/%m/%Y')
        end_of_double = df4.last_valid_index()
        end_of_double1 = df4.last_valid_index()
        date_range2 = pd.date_range(start = second_double_date, end = end_of_double1, freq = '12MS')
        df4.loc[date_range2, 'Payments'] = df4.loc[date_range2, 'Payments'] * int(factor)
        inc_again = input('Do the ground rent payments increase again (yes/no)?')
        if inc_again == 'no':
            print ('No futher increases noted')
        elif inc_again == 'yes':
            second_factor = input('Please enter the factor by which your already increased ground rents are set to increase by: ' )
            second_increase_date = input('Please enter the date of you will receive the first receipt of your second increased ground rent payment (DD/MM/YYYY): ')
            second_increase_date = pd.to_datetime(second_increase_date, format='%d/%m/%Y')
            end_of_double = df2.last_valid_index()
            date_range3 = pd.date_range(start = second_increase_date, end = end_of_double, freq = '12MS')
            df2.loc[date_range3, 'Payments'] = df2.loc[date_range3, 'Payments'] * float(second_factor)
            
            second2_double_date = input('Please enter the date of you will receive the second receipt of your second increased ground rent payment (DD/MM/YYYY): ')
            second2_double_date = pd.to_datetime(second2_double_date, format='%d/%m/%Y')
            end_of_double1 = df4.last_valid_index()
            date_range2 = pd.date_range(start = second2_double_date, end = end_of_double1, freq = '12MS')
            df4.loc[date_range2, 'Payments'] = df4.loc[date_range2, 'Payments'] * float(second_factor)

    df2['Payments Batch #1'] = df2['Payments Batch #1']  + risk_premium
    df4['Payments Batch #2'] = df4['Payments Batch #2']  + risk_premium
    df2['Payments'] = df2['Payments'].astype(int)
    df4['Payments'] = df4['Payments'].astype(int)
    num = df2['Payments'].tolist()
    rate_lists = df2['Payments Batch #1'].tolist()
    num1 = df4['Payments'].tolist()
    rate_lists1 = df4['Payments Batch #2'].tolist()
    def npv(num, rate_lists):
        result = 0
        for t,(num,rate_lists) in enumerate(zip(num,rate_lists),1):
            result += (num / (1+(rate_lists/100)) ** t)
        return result
    npv1 = npv(num,rate_lists)
    def npv(num1, rate_lists1):
        result = 0
        for t,(num1,rate_lists1) in enumerate(zip(num1,rate_lists1),1):
            result += (num1 / (1+(rate_lists1/100)) ** t)
        return result
    npv2 = npv(num1,rate_lists1)
    print (npv1 + npv2)
else:
    print ('Please enter either 1 or 2')

C:\Users\harry\AppData\Local\Temp\ipykernel_1636\1657220187.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(futher_dates)
C:\Users\harry\AppData\Local\Temp\ipykernel_1636\1657220187.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df4.append(futher_dates)


297690.75287103315


In [38]:
# This section is dedicated to calculatingh the npv of the additional costs to be expected in the future 
range1 = int(input('How many additional costs are you expecting for this investment, if none enter 0: '))
if range1 < 1:
    print ('No additonal costs noted')
    adcostnpvsum = 0
elif range1 >= 1:
    vals = [int(input('Please enter the value of the additonal costs in time assending order: ')) for _ in range(range1)]
    yrs = [input('Please enter the date you expect each payment to occur in time assending order given as: ') for _ in range(range1)]
    date_range = pd.date_range(start = first_payment_date, end = end_date, freq = '12MS')
    date_range = date_range.to_frame(index= False)
    date_range.columns = ['Years']
    date_range.index += 1
    #
    yrs = pd.to_datetime(yrs, format='%d/%m/%Y')
    n1 = date_range['Years'].last_valid_index()
    date_range['Period'] = range(1, n1 +1)
    date_range = date_range.set_index(pd.DatetimeIndex(date_range['Years']))
    #
    periods = date_range.loc[yrs, 'Period'].to_frame()
    periods.dtypes
    periods1 = periods['Period'].tolist()
    #
    vals = np.array(vals)
    powers = np.array(periods1)
    rates2 = np.array(z2.loc[yrs, 'Yield'].tolist())
    print (rates2)
    print (vals)
    print (powers)
    #
    def adcostnpv(vals, rates2, periods):
        for i in (vals, rates2, periods):
            result = 0 
            result += (vals / ((1+(rates2/100)) ** powers))
        return result
    print (adcostnpv(vals, rates2, periods))
    np.sum(adcostnpv(vals, rates2, periods))
    adcostnpvsum = np.sum(adcostnpv(vals, rates2, periods))
    print (adcostnpvsum)
else:
    print ('please follow the format instructions given')

No additonal costs noted


In [41]:
NPVsum = npv1 + npv2 - adcostnpvsum
print('The Net Presnt Value of the additional costs are','£', round(adcostnpvsum,2))
print()
print('The Net Presnt Value of the January payments are','£', round(npv1,2))
print()
print('The Net Presnt Value of the June payments are','£', round(npv2,2))
print()
print('The Net Presnt Value of the ground rent investment is','£',round(NPVsum,2))

The Net Presnt Value of the additional costs are £ 0

The Net Presnt Value of the January payments are £ 91808.18

The Net Presnt Value of the June payments are £ 78206.01

The Net Presnt Value of the ground rent investment is £ 170014.19


In [42]:
import tkinter as tk

master = tk.Tk()

def write_slogan():
    info_message = round(NPVsum,2)
    tk.Label(master, text=info_message).grid(row=2, column=1) 

btn = tk.Button(master, text='Your freehold investments NPV', command=write_slogan)
btn.grid(row=3, column=1, sticky=tk.W, pady=4)

master.mainloop()